# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1d849a26d0>
├── 'a' --> tensor([[ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864]])
└── 'x' --> <FastTreeValue 0x7f1d849a2d30>
    └── 'c' --> tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                        [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                        [ 8.4384e-01, -4.4212e-01,  5.6725e-01, -1.8417e-03]])

In [4]:
t.a

tensor([[ 0.8990,  0.5965, -1.6339],
        [ 0.2272,  0.2877,  0.7864]])

In [5]:
%timeit t.a

69.2 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1d849a26d0>
├── 'a' --> tensor([[-0.1251, -0.8949,  0.3470],
│                   [ 1.5592, -0.1718,  0.5558]])
└── 'x' --> <FastTreeValue 0x7f1d849a2d30>
    └── 'c' --> tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                        [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                        [ 8.4384e-01, -4.4212e-01,  5.6725e-01, -1.8417e-03]])

In [7]:
%timeit t.a = new_value

89.6 ns ± 0.959 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864]]),
    x: Batch(
           c: tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                      [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                      [ 8.4384e-01, -4.4212e-01,  5.6725e-01, -1.8417e-03]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8990,  0.5965, -1.6339],
        [ 0.2272,  0.2877,  0.7864]])

In [11]:
%timeit b.a

64.3 ns ± 0.878 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8781, -0.0678, -1.9473],
               [-0.9390, -0.9554, -2.2939]]),
    x: Batch(
           c: tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                      [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                      [ 8.4384e-01, -4.4212e-01,  5.6725e-01, -1.8417e-03]]),
       ),
)

In [13]:
%timeit b.a = new_value

630 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 26 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

282 µs ± 9.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 5.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1d58dcda30>
├── 'a' --> tensor([[[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]],
│           
│                   [[ 0.8990,  0.5965, -1.6339],
│                    [ 0.2272,  0.2877,  0.7864]]])
└── 'x' --> <FastTreeValue 0x7f1d8499b8b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.7 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1cd7687700>
├── 'a' --> tensor([[ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864],
│                   [ 0.8990,  0.5965, -1.6339],
│                   [ 0.2272,  0.2877,  0.7864]])
└── 'x' --> <FastTreeValue 0x7f1cd7687190>
    └── 'c' --> tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                        [ 1.3546e+00,  7.6471e-01,  1.1446e-01,

In [23]:
%timeit t_cat(trees)

48.1 µs ± 4.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.1 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]],
       
               [[ 0.8990,  0.5965, -1.6339],
                [ 0.2272,  0.2877,  0.7864]]]),
    x: Batch(
           c: tensor([[[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                       [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                       [ 8.4384e-01, -4

In [26]:
%timeit Batch.stack(batches)

105 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864],
               [ 0.8990,  0.5965, -1.6339],
               [ 0.2272,  0.2877,  0.7864]]),
    x: Batch(
           c: tensor([[-1.8607e+00,  1.0264e-01, -8.9569e-01,  4.3114e-01],
                      [ 1.3546e+00,  7.6471e-01,  1.1446e-01, -7.3984e-01],
                      [ 8.4384e-01, -4.4212e-01,  5.6725e-01, -1.8417e-03],
                      [-1.8607e+00,  1.0264e-

In [28]:
%timeit Batch.cat(batches)

187 µs ± 3.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

593 µs ± 38.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
